In [1]:
# Importanto modulos
from IPython.display import Image # para visualizar imagem/mapa
import ee # Ferramentas Earth Engine
from pprint import pprint # para fazer print com facil leitura


In [2]:
# Iniciando conexao com EarthEngine
ee.Initialize()

In [3]:
# Carregando SRTM do Asset
image = ee.Image('srtm90_v4')
pprint(image)
# acessando info da imagem
pprint(image.getInfo())

{'bands': [{'crs': 'EPSG:4326',
            'crs_transform': [0.000833333333333,
                              0.0,
                              -180.0,
                              0.0,
                              -0.000833333333333,
                              60.0],
            'data_type': {'max': 32767,
                          'min': -32768,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensions': [432000, 144000],
            'id': 'elevation'}],
 'id': 'srtm90_v4',
 'properties': {'system:asset_size': 18827626666,
                'system:time_end': 951177600000,
                'system:time_start': 950227200000},
 'type': 'Image',
 'version': 1494271934303000}


In [4]:
# visualizando Thumb
Image(url=ee.Image('USGS/SRTMGL1_003').getThumbUrl({'min': 0, 'max': 3000}))


In [5]:
# definindo um ponto
xy = ee.Geometry.Point([86.9250, 27.9881])
# amostrando a elevação no ponto criado
elev = image.sample(xy, 30).first().get('elevation').getInfo()
# Valor da elevacao
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8794


In [6]:
# Criando mapa dinamico (webmap) com folium
import folium

In [7]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, eeImageObject, visParams, name):
  mapID = ee.Image(eeImageObject).getMapId(visParams)
  folium.raster_layers.TileLayer(
    tiles = "https://earthengine.googleapis.com/map/"+mapID['mapid']+
      "/{z}/{x}/{y}?token="+mapID['token'],
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)



In [8]:
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer


In [9]:
# definindo parametros de visualizacao.
visParams = {'min':0, 'max':3000, 'palette':['225ea8','41b6c4','a1dab4','ffffcc']}


In [10]:
# Criando folium map
myMap = folium.Map(location=[20, 0], zoom_start=3, height=500)


In [11]:
# Adicionando elevacao ao mapa
myMap.add_ee_layer(image, visParams, 'DEM')



In [12]:
# Add a layer control panel to the map.
myMap.add_child(folium.LayerControl())


In [13]:
# Display the map.
display(myMap)